# EDA - Ámbitos Electorales y Locales de Votación

🔧 **Instrucciones**: reemplazá las URLs de abajo por los enlaces **RAW** de tus archivos en GitHub.

- Si **tenés CSV** todavía, pegá sus URLs y el notebook los convertirá a Parquet.
- Si ya **tenés Parquet**, pegá directamente las URLs de Parquet y salteá la conversión.

> En GitHub abrí el archivo → botón **Raw** → copiá esa URL.


In [ ]:
!pip -q install pyarrow pandas matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ⬇️ Pegar acá las URLs RAW (CSV o Parquet)
AMBITOS_CSV_URL   = "https://raw.githubusercontent.com/USUARIO/REPO/RAMA/ambitosElectorales.csv"  # opcional si convertís
LOCALES_CSV_URL   = "https://raw.githubusercontent.com/USUARIO/REPO/RAMA/localesDeVotacionyMesas.csv"  # opcional si convertís

# Si YA tenés los Parquet, podés usarlos directo:
AMBITOS_PARQUET_URL = ""  # ej: https://raw.githubusercontent.com/USUARIO/REPO/RAMA/ambitosElectorales.parquet
LOCALES_PARQUET_URL = ""  # ej: https://raw.githubusercontent.com/USUARIO/REPO/RAMA/localesDeVotacionyMesas.parquet

# Carpeta local de trabajo en Colab
workdir = Path("/content/electorales")
workdir.mkdir(parents=True, exist_ok=True)
(workdir / "data").mkdir(exist_ok=True)
parquet_dir = workdir / "data"

def maybe_read_csv(url):
    if not url:
        return None
    try:
        return pd.read_csv(url)
    except Exception:
        return pd.read_csv(url, sep=None, engine="python", encoding="utf-8", on_bad_lines="skip")

def read_parquet_or_none(url):
    if not url:
        return None
    return pd.read_parquet(url, engine="pyarrow")

# 1) Intentar leer Parquet si se proporcionaron URLs
ambitos = read_parquet_or_none(AMBITOS_PARQUET_URL)
locales = read_parquet_or_none(LOCALES_PARQUET_URL)

# 2) Si no hay Parquet, leer CSV y convertir a Parquet
if ambitos is None and AMBITOS_CSV_URL:
    ambitos_csv = maybe_read_csv(AMBITOS_CSV_URL)
    ambitos_parquet_path = parquet_dir / "ambitosElectorales.parquet"
    ambitos_csv.to_parquet(ambitos_parquet_path, index=False)
    ambitos = pd.read_parquet(ambitos_parquet_path)

if locales is None and LOCALES_CSV_URL:
    locales_csv = maybe_read_csv(LOCALES_CSV_URL)
    locales_parquet_path = parquet_dir / "localesDeVotacionyMesas.parquet"
    locales_csv.to_parquet(locales_parquet_path, index=False)
    locales = pd.read_parquet(locales_parquet_path)

ambitos.shape, locales.shape

In [ ]:
# 🧭 Vistas rápidas
display(ambitos.head(5))
display(locales.head(5))

print("Ambitos dtypes:\n", ambitos.dtypes)
print("\nLocales dtypes:\n", locales.dtypes)

In [ ]:
# 🔎 Nulos por columna (top 20)
ambitos_nulls = ambitos.isnull().sum().sort_values(ascending=False).head(20)
locales_nulls = locales.isnull().sum().sort_values(ascending=False).head(20)
print("Ambitos nulos top 20:\n", ambitos_nulls)
print("\nLocales nulos top 20:\n", locales_nulls)

In [ ]:
# 📊 Ejemplo de gráfica simple (una sola figura, sin estilos específicos)
cat_cols = [c for c in ambitos.columns if ambitos[c].dtype == 'object']
if cat_cols:
    col = cat_cols[0]
    ambitos[col].value_counts().head(10).plot(kind='bar')
    plt.title(f"Top 10 valores de '{col}' (ambitos)")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

## Próximos pasos
Pegá acá la **consigna del EDA** y vamos extendiendo el notebook con joins, limpieza, reglas de calidad de datos, KPIs y visualizaciones.
